In [44]:
import pandas as pd
import numpy as np
from scipy import stats
import regression as reg

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso, Ridge

In [45]:
ldn=pd.read_csv('londonairbnb_cleaned.csv')

In [46]:
ldn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85257 entries, 0 to 85256
Data columns (total 58 columns):
id                                              85257 non-null int64
last_scraped                                    85257 non-null object
experiences_offered                             85257 non-null object
host_since                                      85257 non-null object
host_is_superhost                               85257 non-null int64
host_verifications                              85257 non-null object
host_has_profile_pic                            85257 non-null int64
host_identity_verified                          85257 non-null int64
neighbourhood                                   85257 non-null object
neighbourhood_cleansed                          85257 non-null object
latitude                                        85257 non-null float64
longitude                                       85257 non-null float64
is_location_exact                               85257

In [47]:
ldn['room_type'].value_counts()

Entire home/apt    47111
Private room       36002
Hotel room          1510
Shared room          634
Name: room_type, dtype: int64

In [48]:
private_room=ldn.loc[ldn['room_type']=='Private room']
private_room.head()

id last_scraped experiences_offered  host_since  host_is_superhost  \
1   13913   2019-09-15            business  2009-11-16                  0   
6   25123   2019-09-15                none  2010-04-05                  1   
8   31361   2019-09-15                none  2010-05-29                  1   
9   33332   2019-09-15                none  2010-06-14                  0   
10  34825   2019-09-15                none  2010-06-22                  0   

                                   host_verifications  host_has_profile_pic  \
1           ['email', 'phone', 'facebook', 'reviews']                     1   
6   ['email', 'phone', 'reviews', 'offline_governm...                     1   
8   ['email', 'phone', 'facebook', 'reviews', 'jum...                     1   
9   ['email', 'phone', 'reviews', 'jumio', 'offlin...                     1   
10                      ['email', 'phone', 'reviews']                     1   

    host_identity_verified    neighbourhood neighbourhood_cleansed  ...  \
1                        0  LB of Islington              Islington  ...   
6                        0     LB of Barnet                 Barnet  ...   
8                        1     LB of Newham                 Newham  ...   
9                        1   LB of Hounslow               Hounslow  ...   
10                       0      St. Pancras                 Camden  ...   

    require_guest_profile_picture  require_guest_phone_verification  \
1                               0                                 0   
6                               0                                 0   
8                               1                                 1   
9                               0                                 0   
10                              0                                 0   

    calculated_host_listings_count  \
1                                3   
6                                3   
8                                2   
9                                1   
10                               1   

   calculated_host_listings_count_entire_homes  \
1                                            1   
6                                            0   
8                                            0   
9                                            0   
10                                           0   

   calculated_host_listings_count_private_rooms  \
1                                             2   
6                                             3   
8                                             2   
9                                             1   
10                                            1   

    calculated_host_listings_count_shared_rooms  reviews_per_month  host_days  \
1                                             0               0.14     3590.0   
6                                             0               1.06     3450.0   
8                                             0               1.03     3396.0   
9                                             0               0.05     3380.0   
10                                            0               1.70     3372.0   

    days_from_last_review amenities_count  
1                    97.0              55  
6                    79.0              17  
8                    77.0              25  
9                   112.0              14  
10                    5.0              12  

[5 rows x 58 columns]

In [49]:
private_room_num = private_room.select_dtypes(["int64","float64"])

private_room_num_cleaned = private_room_num[(np.abs(stats.zscore(private_room_num)) < 3).all(axis = 1)]
private_room_num_cleaned.describe()

id  host_is_superhost  host_has_profile_pic  \
count  2.766100e+04       27661.000000               27661.0   
mean   2.250307e+07           0.162069                   1.0   
std    1.048260e+07           0.368521                   0.0   
min    2.512300e+04           0.000000                   1.0   
25%    1.438056e+07           0.000000                   1.0   
50%    2.243234e+07           0.000000                   1.0   
75%    3.192041e+07           0.000000                   1.0   
max    3.861236e+07           1.000000                   1.0   

       host_identity_verified      latitude     longitude  is_location_exact  \
count            27661.000000  27661.000000  27661.000000       27661.000000   
mean                 0.343552     51.509850     -0.117567           0.606486   
std                  0.474902      0.052799      0.101793           0.488538   
min                  0.000000     51.352470     -0.434150           0.000000   
25%                  0.000000     51.475660     -0.186790           0.000000   
50%                  0.000000     51.515470     -0.109470           1.000000   
75%                  1.000000     51.544570     -0.054920           1.000000   
max                  1.000000     51.667320      0.194600           1.000000   

       accommodates     bathrooms      bedrooms  ...  \
count  27661.000000  27661.000000  27661.000000  ...   
mean       1.797296      1.133057      0.989191  ...   
std        0.614610      0.295148      0.229256  ...   
min        1.000000      0.500000      0.000000  ...   
25%        1.000000      1.000000      1.000000  ...   
50%        2.000000      1.000000      1.000000  ...   
75%        2.000000      1.000000      1.000000  ...   
max        4.000000      2.000000      2.000000  ...   

       require_guest_profile_picture  require_guest_phone_verification  \
count                        27661.0                           27661.0   
mean                             0.0                               0.0   
std                              0.0                               0.0   
min                              0.0                               0.0   
25%                              0.0                               0.0   
50%                              0.0                               0.0   
75%                              0.0                               0.0   
max                              0.0                               0.0   

       calculated_host_listings_count  \
count                    27661.000000   
mean                         2.912078   
std                          5.387387   
min                          1.000000   
25%                          1.000000   
50%                          1.000000   
75%                          2.000000   
max                         54.000000   

       calculated_host_listings_count_entire_homes  \
count                                 27661.000000   
mean                                      0.371642   
std                                       1.636217   
min                                       0.000000   
25%                                       0.000000   
50%                                       0.000000   
75%                                       0.000000   
max                                      28.000000   

       calculated_host_listings_count_private_rooms  \
count                                  27661.000000   
mean                                       2.502693   
std                                        4.404129   
min                                        1.000000   
25%                                        1.000000   
50%                                        1.000000   
75%                                        2.000000   
max                                       38.000000   

       calculated_host_listings_count_shared_rooms  reviews_per_month  \
count                                 27661.000000       27661.000000   
mean                                      0.0045

In [50]:
abs(private_room_num_cleaned.corr())["price"].sort_values(ascending = False).head()

price                     1.000000
accommodates              0.157693
availability_30           0.152132
review_scores_value       0.138343
review_scores_accuracy    0.134530
Name: price, dtype: float64

In [51]:
abs(private_room_num_cleaned.corr())["calculated_host_listings_count"].sort_values(ascending = False).head()


calculated_host_listings_count                  1.000000
calculated_host_listings_count_private_rooms    0.949533
calculated_host_listings_count_entire_homes     0.641408
cleaning_fee                                    0.177222
reviews_per_month                               0.173120
Name: calculated_host_listings_count, dtype: float64

In [52]:
col_corr=['id','beds', 'guests_included', 'availability_60', 'availability_90', 'availability_365', 
          'days_from_last_review','review_scores_rating','review_scores_accuracy', 'review_scores_checkin', 
          'review_scores_communication', 'review_scores_location', 'review_scores_cleanliness', 'reviews_per_month', 
          'number_of_reviews_ltm', 'number_of_reviews', 'amenities_count', 'host_is_superhost', 'days_from_last_review',
         'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_entire_homes']
len(col_corr)

21

In [53]:
prv_room_checked_multicorr=private_room_num_cleaned.drop(col_corr, axis=1)
abs(prv_room_checked_multicorr.corr())["price"].sort_values(ascending = False)



price                                          1.000000
accommodates                                   0.157693
availability_30                                0.152132
review_scores_value                            0.138343
cleaning_fee                                   0.099517
calculated_host_listings_count                 0.089291
minimum_nights                                 0.053178
longitude                                      0.049978
is_location_exact                              0.048519
monthly_price                                  0.047017
weekly_price                                   0.045553
bathrooms                                      0.035877
host_days                                      0.032673
host_identity_verified                         0.031067
extra_people                                   0.025447
latitude                                       0.020998
security_deposit                               0.017926
calculated_host_listings_count_shared_rooms    0

In [54]:
selected_feat_first_iteration=private_room_num_cleaned[['accommodates','availability_30','review_scores_value','cleaning_fee',
                            'calculated_host_listings_count','minimum_nights','longitude','is_location_exact',
                            'monthly_price','weekly_price','bathrooms','host_days','host_identity_verified',
                            'extra_people','latitude','security_deposit','instant_bookable','host_has_profile_pic',
                           'requires_license','require_guest_profile_picture','require_guest_phone_verification']]
X = selected_feat_first_iteration
y = private_room_num_cleaned["price"]

In [55]:
# Polynomial Regression Model
X_train, X_test, y_train, y_test = reg.preprocess(X, y)

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

poly=PolynomialFeatures(degree=3)
poly_x_train=poly.fit_transform(X_train)

reg_poly=LinearRegression().fit(poly_x_train,y_train)
poly_x_test=poly.transform(X_test)

train_R2 = reg_poly.score(poly_x_train, y_train)
test_R2 = reg_poly.score(poly_x_test, y_test)
train_mse = mean_squared_error(y_train, reg_poly.predict(poly_x_train))
test_mse = mean_squared_error(y_test, reg_poly.predict(poly_x_test))
coefs = reg_poly.coef_
intercept = reg_poly.intercept_

print(train_R2, test_R2, train_mse, test_mse, coefs, intercept)

0.3568112842277319 0.27931936439628224 894.6334920118697 1053.9752158640015 [-3.22885552e+08  2.88762270e+11  1.77990311e+11 ...  0.00000000e+00
  0.00000000e+00  0.00000000e+00] -263827271013.90872


In [65]:
X1 = private_room_num_cleaned.drop('price', axis=1)

In [66]:
# Linear Regression Model
X_train1, X_test1, y_train, y_test = reg.preprocess(X1, y)


train_R2, test_R2, train_mse, test_mse, coefs, intercept = reg.run_linear(X_train1, X_test1, y_train, y_test)

print('Training r^2:', train_R2)
print('Testing r^2:', test_R2)
print('Training MSE:', train_mse)
print('Testing MSE:', test_mse)


Training r^2: 0.11676526437606882
Testing r^2: 0.10435532472767217
Training MSE: 1228.5218263642419
Testing MSE: 1309.8552164743712


In [67]:
#LASSO model
lasso_table = reg.run_lasso(X_train1, X_test1, y_train, y_test)
lasso_table.head()

alpha  training_r^2    mse_train  testing_r^2     mse_test  intercept  \
0  0.000000      0.116765  1228.521827     0.104355  1309.855690  50.728076   
1  0.201005      0.110192  1237.664922     0.098388  1318.581714  50.728076   
2  0.402010      0.108204  1240.430380     0.095984  1322.098465  50.728076   
3  0.603015      0.105719  1243.886632     0.093458  1325.791954  50.728076   
4  0.804020      0.102787  1247.964788     0.090555  1330.038225  50.728076   

                                        coefficients  
0  [0.5934168823202249, 0.7385138753741953, 0.0, ...  
1  [0.38908631707208513, 0.41873701878221, 0.0, -...  
2  [0.25693156541425805, 0.03594392940554717, 0.0...  
3  [0.16354776281990802, 0.0, 0.0, -0.0, -0.22219...  
4  [0.06804378572951289, 0.0, 0.0, -0.0, -0.04811...

In [68]:
#Ridge model

ridge_table = reg.run_ridge(X_train1, X_test1, y_train, y_test)
ridge_table.head()

alpha  training_r^2    mse_train  testing_r^2     mse_test  intercept  \
1  0.201005      0.116765  1228.521833     0.104355  1309.855789  50.728076   
2  0.402010      0.116765  1228.521853     0.104355  1309.856371  50.728076   
3  0.603015      0.116765  1228.521887     0.104354  1309.856963  50.728076   
4  0.804020      0.116765  1228.521933     0.104354  1309.857565  50.728076   
5  1.005025      0.116765  1228.521993     0.104353  1309.858176  50.728076   

                                        coefficients  
1  [0.5935439056255964, 0.7385122869398388, 0.0, ...  
2  [0.5936331944372939, 0.7384856472616883, 0.0, ...  
3  [0.5937222296986313, 0.7384590271840606, 0.0, ...  
4  [0.5938110123507615, 0.7384324267523061, 0.0, ...  
5  [0.5938995433297274, 0.7384058460107381, 0.0, ...